## SQL IPYNB

# This notebook is a SQL notebook that can be run in Jupyter Notebook or JupyterLab.
# It will be used to queury the columns for our project.

In [6]:
# Install required libraries (only once)
#!pip install wbdata pandas

import wbdata
import pandas as pd
import sqlite3

# 1. Set indicators
indicators = {
    'SP.DYN.LE00.IN': 'life_expectancy',
    'SH.DYN.MORT': 'under5_mortality_rate',
    'SP.ADO.TFRT': 'adolescent_fertility_rate'
}

# 2. Fetch data (NO date filter)
df = wbdata.get_dataframe(indicators)

# 3. Reset index
df = df.reset_index()

# 4. Drop rows with missing values
df = df.dropna()

# 5. Load into SQLite in-memory database
conn = sqlite3.connect(':memory:')
df.to_sql('population_data', conn, index=False, if_exists='replace')

# 6. Create cleaned table
conn.execute("""
CREATE TABLE cleaned_population_data AS
SELECT
    country,
    date,
    life_expectancy,
    under5_mortality_rate,
    adolescent_fertility_rate
FROM
    population_data
WHERE
    life_expectancy IS NOT NULL
    AND under5_mortality_rate IS NOT NULL
    AND adolescent_fertility_rate IS NOT NULL
""")
conn.commit()

# 7. Descriptive stats
stats_query = """
SELECT
    AVG(life_expectancy) AS avg_life_expectancy,
    MIN(life_expectancy) AS min_life_expectancy,
    MAX(life_expectancy) AS max_life_expectancy,
    AVG(under5_mortality_rate) AS avg_under5_mortality,
    MIN(under5_mortality_rate) AS min_under5_mortality,
    MAX(under5_mortality_rate) AS max_under5_mortality,
    AVG(adolescent_fertility_rate) AS avg_adolescent_fertility,
    MIN(adolescent_fertility_rate) AS min_adolescent_fertility,
    MAX(adolescent_fertility_rate) AS max_adolescent_fertility
FROM
    cleaned_population_data
"""
stats_result = pd.read_sql_query(stats_query, conn)
print(stats_result)

# 8. Group by country
grouped_query = """
SELECT
    country,
    AVG(life_expectancy) AS avg_life_expectancy,
    AVG(under5_mortality_rate) AS avg_under5_mortality_rate,
    AVG(adolescent_fertility_rate) AS avg_adolescent_fertility_rate
FROM
    cleaned_population_data
GROUP BY
    country
ORDER BY
    avg_life_expectancy DESC
"""
grouped_result = pd.read_sql_query(grouped_query, conn)
print(grouped_result.head())


# Query the cleaned table into a DataFrame
cleaned_df = pd.read_sql_query("SELECT * FROM cleaned_population_data", conn)

# Now cleaned_df is a normal DataFrame you can manipulate
print(cleaned_df.head())



   avg_life_expectancy  min_life_expectancy  max_life_expectancy  \
0            65.252508               10.989               86.372   

   avg_under5_mortality  min_under5_mortality  max_under5_mortality  \
0             71.114946                   1.4                 767.4   

   avg_adolescent_fertility  min_adolescent_fertility  \
0                 72.515224                     0.512   

   max_adolescent_fertility  
0                   229.604  
                     country  avg_life_expectancy  avg_under5_mortality_rate  \
0                     Monaco            82.924282                   5.166667   
1                    Andorra            82.030308                   7.423077   
2                 San Marino            81.448641                   6.107692   
3                  Euro area            79.545286                   5.312769   
4  Post-demographic dividend            78.542056                   6.698336   

   avg_adolescent_fertility_rate  
0                      14.336